# Multipair ATR indicator breakout grid search example

- Based on `multipair-breakout-atr-slow` individual backtest
- Does grid search over the strategy parameters
  - See `Parameters` class
  - Instead of passing individual parameters we are passing list of options
- Grid search is a lot of CPU work: Running this notebooks depends on how many CPUs your computer has available.
  - Running grid search is not suitable for free Github Codespaces instances.
  - Running this grid search will take **several hours** on 32 CPU core machine
  - Limit the number of parameters for a faster grid search
- Grid search output is different
  - Instead of showing individual results, we show multiple results with summaries of CAGR, max drawdown, Sharpe and such in a table
  - We can sbow all grid search results, but rendering long tables is often slow and hard to read
  - We have functions to sort out and pick the best results by a parameter (usually CAGR, Sharpe)
  - We will also visualise the equity curve of the best result

We explore two candidates
- One optimised for absolute return (CAGR)
- One optimised for risk-adjusted return (Sharpe)
- You can see that the equity curve differs greatly based on which aspect you are optimising for (more severe breakouts only, less time in market)

**Note**: Table scrollbars and color coding missing when viewed on Github.

# Set up

Set up Trading Strategy data client.

In [ ]:
from tradeexecutor.utils.notebook import setup_charting_and_output
from tradingstrategy.client import Client
from tradeexecutor.utils.notebook import setup_charting_and_output, OutputMode

client = Client.create_jupyter_client()

# Set up drawing charts in interactive vector output mode.
# This is slower. See the alternative commented option below.
# setup_charting_and_output(OutputMode.interactive)

# Set up rendering static PNG images.
# This is much faster but disables zoom on any chart.
setup_charting_and_output(OutputMode.static, image_format="png", width=1500, height=1000)

# Parameters

- Strategy parameters define the fixed and grid searched parameters

In [ ]:
from tradingstrategy.chain import ChainId
import datetime

from tradeexecutor.strategy.default_routing_options import TradeRouting
from tradingstrategy.timebucket import TimeBucket
from tradeexecutor.strategy.cycle import CycleDuration
from tradeexecutor.strategy.parameters import StrategyParameters


class Parameters:
    """Parameteres for this strategy.

    - Collect parameters used for this strategy here

    - Both live trading and backtesting parameters
    """

    id = "grid-eth-breakout-atr-1h" # Used in cache paths

    cycle_duration = CycleDuration.cycle_1h  
    candle_time_bucket = TimeBucket.h1
    allocation = 0.98   

    atr_length = [10, 20, 40, 60]   # Bars
    fract = [0.75, 1.0, 1.5, 2, 3, 5, 7]  # Fraction between last hourly close and breakout level

    adx_length = [14, 25, 40, 60]  # Days
    adx_filter_threshold = [15, 25, 35]
    
    trailing_stop_loss_pct = [0.975, 0.98] 
    trailing_stop_loss_activation_level = [1.025, 1.05, 1.075]
    stop_loss_pct = [0.98, 0.95]

    #
    # Live trading only
    #
    chain_id = ChainId.polygon
    routing = TradeRouting.default  # Pick default routes for trade execution
    required_history_period = datetime.timedelta(hours=200)

    #
    # Backtesting only
    #
    backtest_start = datetime.datetime(2022, 1, 1)
    backtest_end = datetime.datetime(2024, 5, 1)
    stop_loss_time_bucket = TimeBucket.m15
    backtest_trading_fee = 0.0005  # Override the default Binance data trading fee and assume we can trade 5 BPS fee on WMATIC-USDC on Polygon on Uniswap v3
    initial_cash = 10_000

parameters = StrategyParameters.from_class(Parameters, grid_search=True) 


# Trading pairs and market data

- Set up our trading pairs
- Load historical market data for backtesting
- We use Binance CEX data so we have longer history to backtest

In [ ]:
from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse, load_partial_data
from tradingstrategy.client import Client
from tradeexecutor.strategy.execution_context import ExecutionContext, notebook_execution_context
from tradeexecutor.utils.binance import create_binance_universe
from tradeexecutor.strategy.universe_model import UniverseOptions

# List of trading pairs we use in the backtest
# In this backtest, we use Binance data as it has more price history than DEXes
trading_pairs = [
    (ChainId.centralised_exchange, "binance", "ETH", "USDT"),
]

def create_trading_universe(
    timestamp: datetime.datetime,
    client: Client,
    execution_context: ExecutionContext,
    universe_options: UniverseOptions,
) -> TradingStrategyUniverse:
    """Create the trading universe.

    - In this example, we load all Binance spot data based on our Binance trading pair list.
    """

    # Backtesting - load Binance data
    strategy_universe = create_binance_universe(
        [f"{p[2]}{p[3]}" for p in trading_pairs],
        candle_time_bucket=Parameters.candle_time_bucket,
        stop_loss_time_bucket=Parameters.stop_loss_time_bucket,
        start_at=universe_options.start_at,
        end_at=universe_options.end_at,
        trading_fee_override=Parameters.backtest_trading_fee,
        forward_fill=True,
    )
    return strategy_universe


strategy_universe = create_trading_universe(
    None,
    client,
    notebook_execution_context,
    UniverseOptions.from_strategy_parameters_class(Parameters, notebook_execution_context)
)


# Indicators

- We use `pandas_ta` Python package to calculate technical indicators
- These indicators are precalculated and cached on the disk
- Indicators are calculated to each pair in our trading pair dataset

In [ ]:
import pandas as pd
import pandas_ta

from tradeexecutor.analysis.regime import Regime
from tradeexecutor.strategy.execution_context import ExecutionContext
from tradeexecutor.strategy.pandas_trader.indicator import IndicatorSet, IndicatorSource
from tradeexecutor.strategy.parameters import StrategyParameters
from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse
from tradingstrategy.utils.groupeduniverse import resample_candles


def daily_price(open, high, low, close) -> pd.DataFrame:
    """Resample pricees to daily for ADX filtering."""
    original_df = pd.DataFrame({
        "open": open,
        "high": high,
        "low": low,
        "close": close,
    })    
    daily_df = resample_candles(original_df, pd.Timedelta(days=1))
    return daily_df


def daily_adx(open, high, low, close, length):
    daily_df = daily_price(open, high, low, close)
    adx_df = pandas_ta.adx(
        close=daily_df.close,
        high=daily_df.high,
        low=daily_df.low,
        length=length,
    )
    return adx_df


def regime(open, high, low, close, length, regime_threshold) -> pd.Series:
    """A regime filter based on ADX indicator.

    Get the trend of BTC applying ADX on a daily frame.
    
    - -1 is bear
    - 0 is sideways
    - +1 is bull
    """
    

    assert open.index.is_monotonic_increasing
    assert high.index.is_monotonic_increasing
    assert low.index.is_monotonic_increasing

    adx_df = daily_adx(open, high, low, close, length)

    def regime_filter(row):
        # ADX, DMP, # DMN
        average_direction_index, directional_momentum_positive, directional_momentum_negative = row.values
        if directional_momentum_positive > regime_threshold:
            return Regime.bull.value
        elif directional_momentum_negative > regime_threshold:
            return Regime.bear.value
        else:
            return Regime.crab.value

    regime_signal = adx_df.apply(regime_filter, axis="columns")    

    assert regime_signal.index.is_monotonic_increasing

    return regime_signal



def create_indicators(
    timestamp: datetime.datetime | None,
    parameters: StrategyParameters,
    strategy_universe: TradingStrategyUniverse,
    execution_context: ExecutionContext
):
    indicators = IndicatorSet()

    # https://github.com/twopirllc/pandas-ta/blob/main/pandas_ta/volatility/atr.py
    indicators.add(
        "atr",
        pandas_ta.atr,
        {"length": parameters.atr_length},
        IndicatorSource.ohlcv,
    )

    # ADX https://www.investopedia.com/articles/trading/07/adx-trend-indicator.asp
    # https://github.com/twopirllc/pandas-ta/blob/main/pandas_ta/trend/adx.py
    indicators.add(
        "adx",
        daily_adx,
        {"length": parameters.adx_length},
        IndicatorSource.ohlcv,
    )

    # Price OHLC resampled to daily
    indicators.add(
        "daily_price",
        daily_price,
        {},
        IndicatorSource.ohlcv,
    )

    # A regime filter to detect the trading pair bear/bull markets
    indicators.add(
        "regime",
        regime,
        {"length": parameters.adx_length, "regime_threshold": parameters.adx_filter_threshold},
        IndicatorSource.ohlcv,
    )
        
    return indicators

# Trading algorithm

- Describe out trading strategy as code

In [ ]:
import logging
from tradeexecutor.state.visualisation import PlotKind
from tradeexecutor.state.trade import TradeExecution
from tradeexecutor.strategy.pandas_trader.strategy_input import StrategyInput, IndicatorDataNotFoundWithinDataTolerance


def decide_trades(
    input: StrategyInput,
) -> list[TradeExecution]:

    # 
    # Decidion cycle setup.
    # Read all variables we are going to use for the decisions.
    #
    parameters = input.parameters
    position_manager = input.get_position_manager()
    state = input.state
    timestamp = input.timestamp
    indicators = input.indicators
    strategy_universe = input.strategy_universe
    cash = position_manager.get_current_cash()

    trades = []

    # Enable trailing stop loss after we reach the profit taking level
    #
    for position in state.portfolio.open_positions.values():
        if position.trailing_stop_loss_pct is None:
            close_price = indicators.get_price(position.pair)
            if close_price >= position.get_opening_price() * parameters.trailing_stop_loss_activation_level:
                position.trailing_stop_loss_pct = parameters.trailing_stop_loss_pct                

    # Trade each of our pair individually.
    #
    # If any pair has open position,
    # do not try to rebalance, but hold that position to the end.
    #
    for pair_desc in trading_pairs:
        pair = strategy_universe.get_pair_by_human_description(pair_desc)

        close_price = indicators.get_price(pair=pair)  # Price the previous 15m candle closed for this decision cycle timestamp
        atr = indicators.get_indicator_value("atr", pair=pair)  # The ATR value at the time of close price
        last_day = (timestamp - parameters.cycle_duration.to_pandas_timedelta()).floor(freq="D")  # POI (point of interest): Account 15m of lookahead bias whehn using decision cycle timestamp
        previous_price = indicators.get_price(timestamp=last_day, pair=pair)  # The price at the start of this hour

        regime_val = indicators.get_indicator_value("regime", pair=pair)  # Because the regime filter is calculated only daily, we allow some lookback
        regime = Regime(regime_val)  # Convert to enum for readability
    
        if None in (atr, close_price, previous_price):
            # Not enough historic data,
            # cannot make decisions yet
            return []
        
        # If regime filter does not have enough data at the start of the backtest,
        # default to bull market
        if regime is None:
            regime = Regime.bull
            

        # We assume a breakout if our current 15m candle has closed
        # above the 1h starting price + (atr * fraction) target level
        long_breakout_entry_level = previous_price + atr * parameters.fract

        # Check for open condition - is the price breaking out
        #
        if not position_manager.is_any_open():
            if regime == Regime.bull:  
                if close_price > long_breakout_entry_level:
                    trades += position_manager.open_spot(
                        pair,
                        value=cash * parameters.allocation,
                        stop_loss_pct=parameters.stop_loss_pct,             
                    )

    # Visualisations
    #
    if input.is_visualisation_enabled():
        visualisation = state.visualisation
        # Visualise ATR for BTC        
        pair = strategy_universe.get_pair_by_human_description(trading_pairs[0])
        visualisation.plot_indicator(
            timestamp, 
            "ATR", 
            PlotKind.technical_indicator_detached, 
            atr,
            pair=pair,
        )

    return trades  # Return the list of trades we made in this cycle

# Grid search

- Run the grid search

In [ ]:
from tradeexecutor.backtest.grid_search import GridCombination, get_grid_search_result_path, perform_grid_search, prepare_grid_combinations

# This is the path where we keep the result files around
storage_folder = get_grid_search_result_path(Parameters.id)

# Popular grid search combinations and indicators for them
combinations = prepare_grid_combinations(
    parameters,
    storage_folder,
    create_indicators=create_indicators,
    strategy_universe=strategy_universe,
)

indicators = GridCombination.get_all_indicators(combinations)

print(f"We prepared {len(combinations)} grid search combinations with total {len(indicators)} indicators which need to be calculated, stored in {storage_folder.resolve()}")

try:
    grid_search_results = perform_grid_search(
        decide_trades,
        strategy_universe,
        combinations,
        trading_strategy_engine_version="0.5",
        multiprocess=True,
    )
except Exception as e:
    print(e)
    print(e.__cause__)

# Search results

In [ ]:
# Set Jupyter Notebook output mode parameters
from tradeexecutor.analysis.grid_search import find_best_grid_search_results
# from tradeexecutor.analysis.grid_search import visualise_table

# Print extension of our backtest
cached_results = [r for r in grid_search_results if r.cached]
print(f"Grid search results available: {len(grid_search_results)}, of which we got cached {len(cached_results)} in {storage_folder}")


## All results

- Show all results in a single table

In [ ]:
from tradeexecutor.analysis.grid_search import analyse_grid_search_result
from tradeexecutor.analysis.grid_search import render_grid_search_result_table

df = analyse_grid_search_result(grid_search_results)
print(f"We have {len(df)} results")

if len(df) < 100:
    render_grid_search_result_table(df)
else:
    print("Too many grid search results to render in a single table")

## Highest profitability (CAGR)

- List of strategy parameter candidates
- From all grid search results, sorted by CAGR (profitability)

In [ ]:
from tradeexecutor.analysis.grid_search import find_best_grid_search_results, render_grid_search_result_table

best_results = find_best_grid_search_results(grid_search_results)
render_grid_search_result_table(best_results.cagr)

The CARG optimised best candidate equity curve.

In [ ]:
from tradeexecutor.visual.grid_search import visualise_single_grid_search_result_benchmark

fig = visualise_single_grid_search_result_benchmark(
    best_results.cagr[0], 
    strategy_universe, 
    initial_cash=Parameters.initial_cash
)
fig.show()

## Highest Sharpe

- List of strategy parameter candidates
- From all grid search results, sorted by Sharpe (risk-adjusted profit)

In [ ]:
render_grid_search_result_table(best_results.sharpe)

The optimised Sharpe result equity curve

In [ ]:
fig = visualise_single_grid_search_result_benchmark(
    best_results.sharpe[0], 
    strategy_universe, 
    initial_cash=Parameters.initial_cash
)
fig.show()